####Name: Aseel Alqash

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_raw=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/day15_real_dataset_large.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          78 non-null     object 
 1   income       77 non-null     float64
 2   city         96 non-null     object 
 3   signup_time  97 non-null     object 
dtypes: float64(1), object(3)
memory usage: 3.2+ KB


In [ ]:
df_raw.head()

,age,income,city,signup_time
0,30,55000.0,NY,2024-01-01
1,NaN,70000.0,SF,2024-01-05
2,45,NaN,LA,not a date
3,unknown,120000.0,NY,2024/02/01
4,28,65000.0,Chicago,2024-01-15


In [ ]:
df_raw.describe()

,income
count,77.000000
mean,77077.922078
std,18780.425303
min,45000.000000
25%,62000.000000
50%,75000.000000
75%,91000.000000
max,120000.000000


In [ ]:
df_raw.head()
df_raw.sample(50)
df_raw.dtypes

,0
age,object
income,float64
city,object
signup_time,object


The dataset was inspected to understand data types, missing values, and potential anomalies before designing the cleaning pipeline.

In [ ]:
df_raw.shape

(97, 4)

### Cleaning Plan:-
The data cleaning process includes converting numeric columns to appropriate data types, imputing missing values using median imputation, handling extreme values using percentile-based capping, standardizing categorical text fields, and parsing date columns into datetime format. This approach ensures data consistency and robustness while preserving all observations.

In [ ]:
metadata_dict = {
    'age': ['Numeric', 'Median Imputation', 'Yes (age_missing)', 'None'],
    'income': ['Numeric', 'Median Imputation', 'Yes (income_missing)', '99th Percentile Cap'],
    'city': ['String', 'Fill with "unknown"', 'No', 'Lowercased & Trimmed'],
    'signup_time': ['Datetime', 'Fill with Min Date', 'Yes (signup_time_missing)', 'Coerced to Datetime']
}

# Converted it to a DataFrame for a clean display
df_plan = pd.DataFrame.from_dict(
    metadata_dict,
    orient='index',
    columns=['Data Type', 'Imputation Strategy', 'Masking Column', 'Outlier/Format Fix']
)
print("Project Metadata Planning:-")
display(df_plan)

Project Metadata Planning:-


,Data Type,Imputation Strategy,Masking Column,Outlier/Format Fix
age,Numeric,Median Imputation,Yes (age_missing),None
income,Numeric,Median Imputation,Yes (income_missing),99th Percentile Cap
city,String,"Fill with ""unknown""",No,Lowercased & Trimmed
signup_time,Datetime,Fill with Min Date,Yes (signup_time_missing),Coerced to Datetime


###The Data Cleaning Function:-

In [ ]:
import pandas as pd
import numpy as np

def clean_data_project(df_input):
    # I need to Work on a copy of the df to avoid changing the original data
    df = df_input.copy()

    # I have to convert age to numeric (if it was an object) and signup_time to date
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    df['signup_time'] = pd.to_datetime(df['signup_time'], errors='coerce')
    df['signup_time_missing'] = df['signup_time'].isna().astype(int)
    default_date = df['signup_time'].min() # this is so that there are no NaT in the date column
    df['signup_time'] = df['signup_time'].fillna(default_date) # we added the dates instead of the NaT


    # 3). Missing Values
    df['age_missing'] = df['age'].isna().astype(int)
    df['age'] = df['age'].fillna(df['age'].median())

    # Logic for Income
    df['income_missing'] = df['income'].isna().astype(int)
    df['income'] = df['income'].fillna(df['income'].median())

    # 4. Outliers (Cap income at 99th percentile)
    upper_limit = df['income'].quantile(0.99)
    df['income'] = df['income'].clip(upper=upper_limit)

    # 5. Strings (Cleaning whitespace and capitalization for 'city')
    # I used fillna first so the string methods don't break on the 1 missing value
    df['city'] = df['city'].fillna('unknown').str.strip().str.lower()

    # 6. Logging
    print("Cleaning Complete: Missing values filled, income capped, strings normalized.")

    return df

# Run the pipeline on the actual dataset
df_clean = clean_data_project(df_raw)

# Show the results
display(df_clean.head())

Cleaning Complete: Missing values filled, income capped, strings normalized.


,age,income,city,signup_time,signup_time_missing,age_missing,income_missing
0,30.0,55000.0,ny,2024-01-01,0,0,0
1,35.0,70000.0,sf,2024-01-05,0,1,0
2,45.0,75000.0,la,2024-01-01,1,0,1
3,35.0,115200.0,ny,2024-01-01,1,1,0
4,28.0,65000.0,chicago,2024-01-15,0,0,0


### Cleaning Decisions:
- I filled missing numbers with the median so that extreme values don't affect the results too much.

- Very large numbers were capped at the 99th percentile to reduce their impact but keep all the data.

- Catagorical columns ( Text ) were cleaned ( made lowercase, removed spaces ) to avoid duplicate categories.

- Dates were converted to a standard format using ( pd.to_datetime ), so all the values could be read and used correctly

In [ ]:
# Created a summary report
print("Project Verification:-")
print(f"\nTotal Rows: {len(df_clean)}")
print("\n1). Missing Values:-")
print(df_clean[['age', 'income', 'city']].isnull().sum())

print("\n2). Data Types:-")
print(df_clean[['age', 'income', 'signup_time']].dtypes)

print("\n3). Mask Columns Created:-")
print(df_clean[['age_missing', 'income_missing','signup_time_missing']].head())

Project Verification:-

Total Rows: 97

1). Missing Values:-
age       0
income    0
city      0
dtype: int64

2). Data Types:-
age                   float64
income                float64
signup_time    datetime64[ns]
dtype: object

3). Mask Columns Created:-
   age_missing  income_missing  signup_time_missing
0            0               0                    0
1            1               0                    0
2            0               1                    1
3            1               0                    1
4            0               0                    0


In [ ]:
# I created a quick comparison table to check the changes and values before and after cleaning them
comparison = pd.DataFrame({
    'Metric': ['Total Missing Values', 'Age Data Type', 'Income Outliers (Max)', 'Messy City Names'],
    'Before': [df_raw.isnull().sum().sum(), df_raw['age'].dtype, df_raw['income'].max(), "Yes (Mixed casing/spaces)"],
    'After': [df_clean.isnull().sum().sum(), df_clean['age'].dtype, df_clean['income'].max(), "No (Clean/lowercase)"]
})

display(comparison) # I used display because it shows the df in a more clearer and cleaner way

,Metric,Before,After
0,Total Missing Values,40,0
1,Age Data Type,object,float64
2,Income Outliers (Max),120000.0,115200.0
3,Messy City Names,Yes (Mixed casing/spaces),No (Clean/lowercase)


### The Downloaded CSV:-

In [ ]:
from google.colab import files

df_clean.to_csv('Day15_End_to_End_Cleaning.csv', index=False)
files.download('Day15_End_to_End_Cleaning.csv')
print("Success! Now the CSV is downloaded to the computer!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Success! Now the CSV is downloaded to the computer!
